# __init__.py
Before Python3.3, every package must contain a `__init__.py`.  
It's run automatically the first time a Python program imports this package.  

Although after Python3.3, `__init__.py` is no longer a must. (Using namespace to import)  
Using `__init__.py` provides a performance advantage.  


## What does __init__.py do?

### 1. Package initialization
All the code in the directory's `__init__.py` is run automatically.  
Can be used to initialize the state  
e.g. initialize file to create required data files, open connections to databases

### 2. Module usability declarations
Declare that a directory is a Python Package  
In fact, `__init__.py` are often empty in practice


## from * statament behavior
You can use `__all__` list in `__init__.py` to define what is exported when this directory is imported using `from *` statement

```python
# c.py

x = 1
y = 2
__all__ = [x]
```

In [1]:
from c import *

print(x)
print(y)   # not imported since it's not in __all__

1


NameError: name 'y' is not defined

---

# Package Relative Imports

How it works is version dependent
- Python2 implicitly searches package directories first on imports
- Python3 requires explicit relative import syntax to making same-package imports
    - This change enhances code readability by making same-package imports more obvious
    
The directory used relative import can no longer be used as executable program.

## Chagnes in Python3
- Absoulte Import
    - Module searching path skips the package's own directory by default.  
    - Imports only check **sys.path**
- Relatvie Import
    - Extend **from** to allow imports searching the package's directory only
    
## Relatvie Import Basics
- Import with dots: **from . import mod**
    - Imports should be relatvie-only to the containing package
    - Won't look for same-named modules elsewhere including **sys.path**
- Import without dots: **import m**, **from m import x**
    - Python2: relative-then-absolute
        - Search the packages's own directoy first and than **sys.path**
    - Python3: absolute-only
        - Skip the containing package and look up **sys.path**
- Relatvie imports apply to **from** only, not **import**
```python
from . import mod1
from .mod1 import name
from .. import mod2
```

In [2]:
from . import a

SystemError: Parent module '' not loaded, cannot perform relative import

### Examples (Python2 vs Python3)

#### Example 1
- Code

```python
# code/pkg/spam.py
import string
print(string)

# code/pkg/string.py
print('Ni' * 8)
```

- Result

```
$ python3
>>> import pkg.spam
<module 'string' from '/usr/Python34/lib/string.py'>

$ python2
>>> import pkg.spam
NiNiNiNiNiNiNiNi
<moduel 'pkg.string' from 'pkg/string.py'>
```

#### Example 2
- Code

```python
# code/string.py
print('string' * 8)

# code/pkg/string.py
print('Ni' * 8)

# code/pkg/spam.py
import string
print(string)
```

- Result

```
$ python3
>>> import pkg.spam
stringstringstringstringstringstringstringstring
<module 'string' from './string.py'>

$ python2
>>> import pkg.spam
NiNiNiNiNiNiNiNi
<module 'pkg.string' from 'pkg/string.pyc'>
```
        
### Absolute import in Python2
```python
# Use absoulte import in Python2
from __future__ import absolute_import 
```


## The Scope of Relative Imports
- Applys to imports within packages only
    - *Normal imports in files not used as part of a package still search the directory containing the top-level script first*
    - Modules referenced by relative imports must exist in the package
    - Package relatvie import syntax is not even allowed in a file not being used as a package


## Pitfalls of Package Relatvie Imports
Must choose a single mode - package(relative imports) or program(simple imports)

```python
from . import mod          # Not allowed in nonpackage in both Python2 and Python3
import mode                # Does not search file's own directory in package in Python3
```

---

# Namespace Package (Python3.3)
An extension to the import model.  
When Python finds directories without finding `__init__.py`, it creates a namespace package that is the virtual concatenation to all found directories having the requested module name.

In [3]:
# ns/mod.py (without __init__.py)

import ns.mod

This is mod.py


In [4]:
ns

<module 'ns' (namespace)>

In [5]:
ns.__path__

_NamespacePath(['C:\\Users\\smszw\\Desktop\\zw-public\\code\\Learning_Python\\Part 5 - Modules and Packages\\ns'])

- Cannot contain an **`__init__.py`** (Since it stops the following algorithm)  
- May span multiple directories that are collected at import time  
- Relative imports work in namespace packages too  

## The Import Algorithm
1. If **`directory/module/__init__.py`** is found, a regular package is imported
2. If **`directory/module.{py, pyc ...}`** is found, a simple module is imported
3. If **`directory/module`** is found and is a directory, it's record and the scan continues with the next directory in the search path
4. If none of the above was found, the scan continues with the next directory in the search path.

If the search path scan completes without returning a module or package by steps 1 or 2, and at least one directory was recorded by step 3, than a namespace package is created.  

Once a namespace package is created, there is no difference between it and a regular package.  

## Optional __init__.py ?
Many packages require no initialization code.  
Thus, they are no longer required after Python3.3  

However, there is performance advantage to have one.  
With namespace packages, all entries in the path must be scanned.  
Compare to it, regular packages stop the algorithm at step 1.  